In [1]:
#DATA PREPARATION, SCROLL DOWN FOR CLUSTERS ONLY
##Open data from SF OpenData
import numpy as np
import csv


def openfile(filename):
    category = []
    date = []
    time = []
    pdistrict = []
    xcoords = []
    ycoords = []
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        reader.next()
        for row in reader:
            category += [row[1]]
            date += [row[4]]
            time += [row[5]]
            pdistrict += [row[6]]
            xcoords += [float(row[9])]
            ycoords += [float(row[10])]            
    return np.array([category, date, time, pdistrict, xcoords, ycoords]).T

    
crimedata = openfile("SFPD_Incidents_-_from_1_January_2003.csv")

import pandas as pd

df = pd.DataFrame(crimedata)

##Create a year and a time an hour variable
year = []
for i in crimedata.T[1]: # for every row in the date
    year.append(int(i.split("/")[-1])) 

df[6] = year # the 6th column of the dataframe is set to the year list

df[7] = df[2].str.split(":").str.get(0)

In [2]:
df[11] = df[7].astype(float)
dn = [] 
for i in df[11]: 
    if i <= 5:
        dn.append(1) #Hours 1-5 (night)
    elif i >5 and i <=11:
        dn.append(2) #Morning
    elif i >11 and i <=14:
        dn.append(3) #Midday 
    elif i >15 and i <=17:
        dn.append(4) #Afternoon
    else:
        dn.append(5) #Hours 18-23 (evening)

        
df[12] = dn ## 12th column of the dataframe now indicates 1 = night, 2 = morning, 3 = midday, 4 = afternoon, 5 = evening



In [3]:
from sklearn.cluster import KMeans
import numpy as np
#Convert the coordinates from strings to floats
df[4] = df[4].astype(float)
df[5] = df[5].astype(float)

# Remove weird coordinates, which are placed far away from the SF area
df_true_coords = df[df[5] != 90]
df_true_coords = df_true_coords[df_true_coords[4] < -122]

df_p = df_true_coords[df_true_coords[0].isin(['PROSTITUTION'])]

In [9]:
#Store the latitude and longtitude coordinates in a numpy array

coordinates = np.array([df_p[4], df_p[5]]).T


#Fitting the cluster model with different number of k-means (2 to 6)
kmeans2 = KMeans(n_clusters=2, random_state=0).fit(coordinates)

kmeans3 = KMeans(n_clusters=3, random_state=0).fit(coordinates)

kmeans4 = KMeans(n_clusters=4, random_state=0).fit(coordinates)

kmeans5 = KMeans(n_clusters=5, random_state=0).fit(coordinates)

kmeans6 = KMeans(n_clusters=6, random_state=0).fit(coordinates)

In [14]:
#The predict command assign each observation to the nearest cluster
df_p["cluster2"] = kmeans2.predict(coordinates)
df_p["cluster3"] = kmeans3.predict(coordinates)
df_p["cluster4"] = kmeans4.predict(coordinates)
df_p["cluster5"] = kmeans5.predict(coordinates)
df_p["cluster6"] = kmeans6.predict(coordinates)

#Don't mind the warnings. It's not a mistake. They can be turned off. 

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

In [15]:
#Give columns reasonable names
df_p.columns = ['CATEGORIES','DATE','TIME','DISTRICT','LON','LAT','YEAR','HOUR','HOUR2','TIMEDAYCAT', 'CLUSTER2','CLUSTER3','CLUSTER4','CLUSTER5','CLUSTER6']


In [18]:
#Prepare data for javaScript. Drop cols
df_p_json = df_p.drop(['CATEGORIES','DATE','TIME','DISTRICT','YEAR','HOUR','TIMEDAYCAT'], axis=1)

#Create a json file with the desired dataset
df_p_json.to_json('prostitutiondata.json', orient='records')

In [19]:
#Make data with the cluster centres. 
k2 = pd.DataFrame(kmeans2.cluster_centers_) #Creates a dataset with number of rows according to number of cluster, columns with latitude and longtitude 
k3 = pd.DataFrame(kmeans3.cluster_centers_)
k4 = pd.DataFrame(kmeans4.cluster_centers_)
k5 = pd.DataFrame(kmeans5.cluster_centers_)
k6 = pd.DataFrame(kmeans6.cluster_centers_)

k2.columns = ["LON", "LAT"]
k3.columns = ["LON", "LAT"]
k4.columns = ["LON", "LAT"]
k5.columns = ["LON", "LAT"]
k6.columns = ["LON", "LAT"]

In [20]:
#Create json files for each dataset, which will be used on the website
k2.to_json('k2.json', orient='records')
k3.to_json('k3.json', orient='records')
k4.to_json('k4.json', orient='records')
k5.to_json('k5.json', orient='records')
k6.to_json('k6.json', orient='records')